In [26]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import math

from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier

from sklearn.neural_network import MLPClassifier

from sklearn import svm


In [27]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
#dataset_path = "/content/drive/MyDrive/TP_DL/olive.csv"
dataset_path = "/content/drive/MyDrive/SRH/Deep_Learning/TP_DL/olive.csv"
df_ol = pd.read_csv(dataset_path)
df_ol.head(2)
print(len(df_ol))
#df_ol['Region'].unique()

572


* configure the dataset :
  * correct the type :

In [29]:
df_ol.loc[df_ol['Region'] == 'North-Apulia', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'South-Apulia', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'Calabria', 'Region'] = 'organic extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'Sicily', 'Region'] = 'organic extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'Inland-Sardinia', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'Coast-Sardinia', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'Umbria', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'East-Liguria', 'Region'] = 'extra virgin olive oil'
df_ol.loc[df_ol['Region'] == 'West-Liguria', 'Region'] = 'extra virgin olive oil'
print(len(df_ol))
print(df_ol['Region'].unique())

572
['extra virgin olive oil' 'organic extra virgin olive oil']


*  
 *  removing unnecessary Fields : 

In [30]:
df_ol=df_ol.drop('id' , axis= 1)
df_ol=df_ol.drop('other' , axis= 1)
df_ol=df_ol.drop('Area' , axis= 1)
df_ol.head(2)
len(df_ol)

572

* data augmentation :



In [31]:
# divide the data : 
df_ol_organic = df_ol[df_ol['Region'] == 'organic extra virgin olive oil']
df_ol_extra = df_ol[df_ol['Region'] == 'extra virgin olive oil']

df_ol_organic.loc[df_ol_organic['Region'] == 'organic extra virgin olive oil', 'Region'] = 1


In [32]:
# Fit the Gaussian mixture model to the data : 
column_names = df_ol_organic.columns.tolist()

gmm = GaussianMixture(n_components=1) # we chose to have 1 container
gmm.fit(df_ol_organic) 

n_samples = len(df_ol_organic) / 2
generated_data = gmm.sample(n_samples)[0]
generated_data = pd.DataFrame(generated_data, columns=column_names)

# concatenate the generated data with your original dataset
df_ol_organic = pd.concat([df_ol_organic, generated_data])

df_ol_organic['Region'] = 'organic extra virgin olive oil'


In [33]:
# Reconcatinate the data : 
df_ol = pd.concat([df_ol_organic, df_ol_extra])
df_ol = df_ol.reset_index(drop=True)
df_ol.head(2)

,Region,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,organic extra virgin olive oil,2.0,1315.0,139.0,230.0,7299.0,832.0,42.0,60.0
1,organic extra virgin olive oil,2.0,1321.0,136.0,217.0,7174.0,950.0,43.0,63.0


* divide data for training and prediction :

In [34]:
# specify your features and target variable
X = df_ol.drop('Region', axis=1)
y = df_ol['Region']

# split the dataset into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_precision = pd.DataFrame(columns=['Algorithm', 'precision'])
 

In [35]:
# #organic
# nb_organic_train = math.floor((2 * len(df_ol_organic)) / 3)
# nb_organic_test = 1 - nb_organic_train
# df_ol_organic_train = df_ol_organic.sample(nb_organic_train)
# random_indices_organic = df_ol_organic_train.index
# df_ol_organic_test = df_ol_organic.drop(index=random_indices_organic)
# df_ol_organic_train = df_ol_organic_train.reset_index(drop=True)
# df_ol_organic_test = df_ol_organic_test.reset_index(drop=True)
# df_ol_organic_train_features = df_ol_organic_train.copy()
# df_ol_organic_train_features.drop('Region', axis=1)

# df_ol_organic_test_features = df_ol_organic_test.copy()
# df_ol_organic_test_features.drop('Region', axis=1)

# df_ol_organic_train_Y = df_ol_organic_train_features['Region']
# df_ol_organic_test_Y = df_ol_organic_test_features['Region']


# #extra
# nb_extra_train = math.floor((2 * len(df_ol_extra)) / 3)
# nb_extra_test = 1 - nb_extra_train
# df_ol_extra_train = df_ol_extra.sample(nb_extra_train)
# random_indices_extra = df_ol_extra_train.index
# df_ol_extra_test = df_ol_extra.drop(index=random_indices_extra)
# df_ol_extra_train = df_ol_extra_train.reset_index(drop=True)
# df_ol_extra_test = df_ol_extra_test.reset_index(drop=True)

# df_ol_extra_train_features = df_ol_extra_train.copy()
# df_ol_extra_train_features.drop('Region', axis=1)

# df_ol_extra_test_features = df_ol_extra_test.copy()
# df_ol_extra_test_features.drop('Region', axis=1)

# df_ol_extra_train_Y = df_ol_extra_train_features['Region']
# df_ol_extra_test_Y = df_ol_extra_test_features['Region']

# df_ol_train_features = pd.concat([df_ol_extra_train_features, df_ol_organic_train_features])
# df_ol_train_Y = pd.concat([df_ol_extra_train_Y, df_ol_organic_train_Y])

# df_ol_test_features = pd.concat([df_ol_extra_test_features, df_ol_organic_test_features])
# df_ol_test_Y = pd.concat([df_ol_extra_test_Y, df_ol_organic_test_Y])

* Naïve Bayesian

In [36]:
# create a Gaussian Naive Bayes classifier object
gnb = GaussianNB()

# train the classifier on the training set
gnb.fit(X_train, y_train)

# make predictions on the testing set
y_pred = gnb.predict(X_test)
pred_df_features = pd.DataFrame(X_test)
pred_df = X_test.copy()
pred_df['Region'] = y_test
pred_df['Region_pred'] = y_pred

pred_df.head(5)

# evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Naive bayes Accuracy:", accuracy)

model_precision = model_precision.append({'Algorithm': 'Naive bayes', 'precision': accuracy, }, ignore_index=True)


Naive bayes Accuracy: 0.9193548387096774


<ipython-input-36-feac3b7f5d4b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'Naive bayes', 'precision': accuracy, }, ignore_index=True)


* Nearest Neighbors (-NN)

In [37]:
# define a range of n_neighbors values to test (we choose k that will give us the value)
k_values = range(1, 20)

# use cross-validation to evaluate each n_neighbors value
validation_scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)  # use 5-fold cross-validation
    validation_scores.append(scores.mean())

# choose the best n_neighbors value
best_k = k_values[validation_scores.index(max(validation_scores))]
print("Best n_neighbors value:", best_k)

# train a new KNN model on the entire dataset using the best n_neighbors value
final_knn_model = KNeighborsClassifier(n_neighbors=best_k)

# Train the model
final_knn_model.fit(X_train, y_train)

# Test the model
y_pred = final_knn_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


model_precision = model_precision.append({'Algorithm': 'Nearest Neighbors', 'precision': accuracy }, ignore_index=True)

Best n_neighbors value: 1
Accuracy: 0.967741935483871


<ipython-input-37-12e1eea1ec93>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'Nearest Neighbors', 'precision': accuracy }, ignore_index=True)


* Linear Discriminate Analysis (LDA)



Linear Discriminant Analysis (LDA) is a technique used in machine learning and statistics to find a linear combination of features that characterizes or separates two or more classes of objects or events. LDA is often used as a dimensionality reduction technique in data preprocessing or feature extraction, as well as a supervised learning algorithm for classification.In our case we'll use it as an algorithm for classification

In [38]:
# create an LDA model and fit it on the training data
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# predict the classes of the testing set using the trained model
y_pred = lda.predict(X_test)

# calculate the accuracy of the model
accuracy = lda.score(X_test, y_test)

print("Accuracy:", accuracy)

model_precision = model_precision.append({'Algorithm': 'LDA', 'precision': accuracy, }, ignore_index=True)

Accuracy: 0.9193548387096774


<ipython-input-38-12c464857723>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'LDA', 'precision': accuracy, }, ignore_index=True)



* Decision Tree


In [39]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

model_precision = model_precision.append({'Algorithm': 'Decision Tree', 'precision': accuracy, }, ignore_index=True)

Accuracy: 0.9354838709677419


<ipython-input-39-6504cadc7672>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'Decision Tree', 'precision': accuracy, }, ignore_index=True)


* Artificial Neural Networks (ANN)


most common parameters of MLPClassifier :
* hidden_layer_sizes: This parameter determines the number of neurons in each layer of the neural network. You can specify a single number to create a single hidden layer with that many neurons, or you can specify a tuple to create multiple hidden layers with the specified number of neurons in each layer.

* activation: This parameter determines the activation function to use for each neuron in the network. The default is 'relu', but you can also choose 'logistic', 'tanh', or 'identity'.

* solver: This parameter determines the optimization algorithm to use to train the neural network. The default is 'adam', but you can also choose 'lbfgs' or 'sgd'.

* alpha: This parameter determines the regularization strength. It is used to help prevent overfitting by adding a penalty term to the error function.

* batch_size: This parameter determines the number of samples to use in each batch during training. Larger batch sizes can help speed up training, but smaller batch sizes may lead to better generalization.

* learning_rate: This parameter determines how the learning rate is adapted during training. The default is 'constant', but you can also choose 'invscaling' or 'adaptive'.

* max_iter: This parameter determines the maximum number of iterations (epochs) to use during training.

* random_state: This parameter determines the random seed used by the random number generator.

Note : For a small dataset with 300 instances, it is generally recommended to start with a simpler network architecture to avoid overfitting. A simple approach to choosing the number of neurons in a single hidden layer is to use a rule of thumb based on the number of input and output neurons. A commonly used rule of thumb is to set the number of neurons in the hidden layer to be the mean of the number of neurons in the input and output layers, or to set it to be some multiple of this value.

hidden_layer_sizesarray-like of shape(n_layers - 2,), default=(100,)
The ith element represents the number of neurons in the ith hidden layer.

activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
Activation function for the hidden layer.

‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x

‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).

‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).

‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

In [40]:
clf = MLPClassifier(hidden_layer_sizes=(70,), max_iter=1000 , activation= 'logistic') # we try different values for the number of layers 10-> 0.24 5 -> 0.75 100 -> 0.8 70 -> 0.91

# Training the classifier on the training data
clf.fit(X_train, y_train)

# Predicting the labels for the test data
y_pred = clf.predict(X_test)

# Calculating the accuracy of the classifier
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

model_precision = model_precision.append({'Algorithm': 'ANN', 'precision': accuracy, }, ignore_index=True)

Accuracy: 0.9032258064516129


<ipython-input-40-7401be267f09>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'ANN', 'precision': accuracy, }, ignore_index=True)


* Support Vector Machine (SVM)

In [41]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", accuracy_score(y_test, y_pred))

model_precision = model_precision.append({'Algorithm': 'SVM', 'precision': accuracy_score(y_test, y_pred), }, ignore_index=True)

Accuracy: 0.9274193548387096


<ipython-input-41-85be198f84ba>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_precision = model_precision.append({'Algorithm': 'SVM', 'precision': accuracy_score(y_test, y_pred), }, ignore_index=True)


* compare presicions :

In [42]:
print(model_precision)

           Algorithm  precision
0        Naive bayes   0.919355
1  Nearest Neighbors   0.967742
2                LDA   0.919355
3      Decision Tree   0.935484
4                ANN   0.903226
5                SVM   0.927419
